In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    elif epoch<50:
        return 0.0001
    else:
        return 0.00005

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    
    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    
    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [5]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.66097, saving model to best_m.h5
 - 6s - loss: 0.5900 - acc: 0.7145 - val_loss: 0.6610 - val_acc: 0.5337
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.4208 - acc: 0.8003 - val_loss: 0.7301 - val_acc: 0.5312
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4232 - acc: 0.8156 - val_loss: 0.7535 - val_acc: 0.6608
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.3634 - acc: 0.8216 - val_loss: 1.2169 - val_acc: 0.6733
Epoch 5/100
Epoch 00005: val_loss improved from 0.66097 to 0.47180, saving model to best_m.h5
 - 4s - loss: 0.3652 - acc: 0.8339 - val_loss: 0.4718 - val_acc: 0.8030
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 3s - loss: 0.3316 - acc: 0.8481 - val_loss: 1.3658 - val_acc: 0.6708
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 3s - loss: 0.3458 - acc: 0.8341 - val_loss: 1.4180 - val_acc: 0.6334
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 3s - loss

Epoch 66/100
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.1314 - acc: 0.9433 - val_loss: 0.3109 - val_acc: 0.8803
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1326 - acc: 0.9433 - val_loss: 0.3178 - val_acc: 0.8903
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1313 - acc: 0.9500 - val_loss: 0.3305 - val_acc: 0.8803
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 4s - loss: 0.1316 - acc: 0.9450 - val_loss: 0.3272 - val_acc: 0.8878
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 4s - loss: 0.1419 - acc: 0.9392 - val_loss: 0.3076 - val_acc: 0.8978
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 4s - loss: 0.1374 - acc: 0.9408 - val_loss: 0.3244 - val_acc: 0.8803
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 4s - loss: 0.1403 - acc: 0.9425 - val_loss: 0.3148 - val_acc: 0.8903
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 4s - loss: 0.1328 - acc: 0.9467 - val_loss: 0.3196 - val_acc: 0.8803
Epoch 74/100
Epo

Epoch 30/100
Epoch 00030: val_loss did not improve
 - 3s - loss: 0.2336 - acc: 0.8964 - val_loss: 0.3677 - val_acc: 0.8603
Epoch 31/100
Epoch 00031: val_loss improved from 0.19498 to 0.19183, saving model to best_m.h5
 - 4s - loss: 0.2382 - acc: 0.9017 - val_loss: 0.1918 - val_acc: 0.9152
Epoch 32/100
Epoch 00032: val_loss improved from 0.19183 to 0.18898, saving model to best_m.h5
 - 4s - loss: 0.2178 - acc: 0.9117 - val_loss: 0.1890 - val_acc: 0.9002
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 3s - loss: 0.2206 - acc: 0.9095 - val_loss: 0.2066 - val_acc: 0.9027
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 3s - loss: 0.2238 - acc: 0.9017 - val_loss: 0.1906 - val_acc: 0.9177
Epoch 35/100
Epoch 00035: val_loss improved from 0.18898 to 0.18696, saving model to best_m.h5
 - 4s - loss: 0.2032 - acc: 0.9167 - val_loss: 0.1870 - val_acc: 0.9152
Epoch 36/100
Epoch 00036: val_loss improved from 0.18696 to 0.18269, saving model to best_m.h5
 - 4s - loss: 0.2515 - acc: 0.8970 

Epoch 94/100
Epoch 00094: val_loss did not improve
 - 3s - loss: 0.1441 - acc: 0.9475 - val_loss: 0.2033 - val_acc: 0.9177
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 3s - loss: 0.1375 - acc: 0.9392 - val_loss: 0.1815 - val_acc: 0.9277
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 3s - loss: 0.1420 - acc: 0.9425 - val_loss: 0.1781 - val_acc: 0.9277
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 3s - loss: 0.1577 - acc: 0.9364 - val_loss: 0.1705 - val_acc: 0.9252
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 3s - loss: 0.1115 - acc: 0.9558 - val_loss: 0.1773 - val_acc: 0.9152
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 3s - loss: 0.1442 - acc: 0.9433 - val_loss: 0.1865 - val_acc: 0.9302
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 3s - loss: 0.1515 - acc: 0.9342 - val_loss: 0.2062 - val_acc: 0.9077
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.73577, saving model to best_m.h5
 - 13s - loss: 0.5305 - acc: 0.7225 - val_loss: 

Epoch 57/100
Epoch 00057: val_loss did not improve
 - 3s - loss: 0.1658 - acc: 0.9325 - val_loss: 0.2363 - val_acc: 0.9102
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 3s - loss: 0.1704 - acc: 0.9283 - val_loss: 0.2809 - val_acc: 0.8878
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 3s - loss: 0.1835 - acc: 0.9289 - val_loss: 0.2276 - val_acc: 0.9127
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 3s - loss: 0.1810 - acc: 0.9275 - val_loss: 0.2364 - val_acc: 0.9102
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 3s - loss: 0.1709 - acc: 0.9247 - val_loss: 0.2297 - val_acc: 0.9027
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 3s - loss: 0.1748 - acc: 0.9317 - val_loss: 0.2295 - val_acc: 0.9152
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 3s - loss: 0.1929 - acc: 0.9100 - val_loss: 0.2370 - val_acc: 0.9152
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 3s - loss: 0.1697 - acc: 0.9231 - val_loss: 0.2560 - val_acc: 0.9127
Epoch 65/100
Epo

Epoch 20/100
Epoch 00020: val_loss improved from 0.22055 to 0.19360, saving model to best_m.h5
 - 4s - loss: 0.2517 - acc: 0.8983 - val_loss: 0.1936 - val_acc: 0.9152
Epoch 21/100
Epoch 00021: val_loss did not improve
 - 3s - loss: 0.2728 - acc: 0.8683 - val_loss: 0.2056 - val_acc: 0.9002
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 3s - loss: 0.2815 - acc: 0.8864 - val_loss: 0.2490 - val_acc: 0.8928
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 3s - loss: 0.2440 - acc: 0.8892 - val_loss: 0.3548 - val_acc: 0.8678
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 3s - loss: 0.2894 - acc: 0.8750 - val_loss: 0.2946 - val_acc: 0.8678
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 3s - loss: 0.2343 - acc: 0.9008 - val_loss: 0.1951 - val_acc: 0.9227
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 3s - loss: 0.2429 - acc: 0.8917 - val_loss: 0.2956 - val_acc: 0.8554
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.2233 - acc: 0.9125 - val_los

Epoch 85/100
Epoch 00085: val_loss did not improve
 - 3s - loss: 0.1647 - acc: 0.9366 - val_loss: 0.1947 - val_acc: 0.9152
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 3s - loss: 0.1480 - acc: 0.9433 - val_loss: 0.1935 - val_acc: 0.9227
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 3s - loss: 0.1740 - acc: 0.9323 - val_loss: 0.1987 - val_acc: 0.9277
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 3s - loss: 0.1725 - acc: 0.9298 - val_loss: 0.1928 - val_acc: 0.9302
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 3s - loss: 0.1448 - acc: 0.9439 - val_loss: 0.2250 - val_acc: 0.9127
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 3s - loss: 0.1648 - acc: 0.9314 - val_loss: 0.1956 - val_acc: 0.9152
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 3s - loss: 0.1395 - acc: 0.9414 - val_loss: 0.1986 - val_acc: 0.9227
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 3s - loss: 0.1567 - acc: 0.9292 - val_loss: 0.1984 - val_acc: 0.9302
Epoch 93/100
Epo

In [6]:
with open('../features/incept_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug2_sub.csv', index=False)

# pre 2110
# new 2048

0.204888837282
         id  is_iceberg
0  5941774d    0.069332
1  4023181e    0.774064
2  b20200e4    0.044102
3  e7f018bb    0.998276
4  4371c8c3    0.390202


In [7]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 128], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 256], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [8]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70436, saving model to best_m.h5
 - 23s - loss: 1.0687 - acc: 0.5899 - val_loss: 0.7044 - val_acc: 0.4663
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 5s - loss: 0.5567 - acc: 0.7372 - val_loss: 1.1984 - val_acc: 0.4663
Epoch 3/100
Epoch 00003: val_loss improved from 0.70436 to 0.58005, saving model to best_m.h5
 - 5s - loss: 0.5339 - acc: 0.7531 - val_loss: 0.5801 - val_acc: 0.6733
Epoch 4/100
Epoch 00004: val_loss improved from 0.58005 to 0.51971, saving model to best_m.h5
 - 5s - loss: 0.4768 - acc: 0.7850 - val_loss: 0.5197 - val_acc: 0.7257
Epoch 5/100
Epoch 00005: val_loss improved from 0.51971 to 0.51253, saving model to best_m.h5
 - 5s - loss: 0.4149 - acc: 0.8081 - val_loss: 0.5125 - val_acc: 0.7257
Epoch 6/100
Epoch 00006: val_loss improved from 0.51253 to 0.46144, saving model to best_m.h5
 - 5s - loss: 0.4054 - acc: 0.8106 - val_loss: 0.4614 - val_acc: 0.8130
Epoch 7/100
Epoch 00007: val_loss did not improve
 

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 5s - loss: 0.1781 - acc: 0.9300 - val_loss: 0.2807 - val_acc: 0.8753
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 5s - loss: 0.2007 - acc: 0.9123 - val_loss: 0.2903 - val_acc: 0.8803
Epoch 64/100
Epoch 00064: val_loss improved from 0.27886 to 0.27883, saving model to best_m.h5
 - 5s - loss: 0.1792 - acc: 0.9275 - val_loss: 0.2788 - val_acc: 0.8753
Epoch 65/100
Epoch 00065: val_loss improved from 0.27883 to 0.27648, saving model to best_m.h5
 - 5s - loss: 0.1935 - acc: 0.9208 - val_loss: 0.2765 - val_acc: 0.8953
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 5s - loss: 0.1828 - acc: 0.9217 - val_loss: 0.2805 - val_acc: 0.8903
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 5s - loss: 0.2194 - acc: 0.9106 - val_loss: 0.3030 - val_acc: 0.8728
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 5s - loss: 0.1978 - acc: 0.9117 - val_loss: 0.3047 - val_acc: 0.8703
Epoch 69/100
Epoch 00069: val_loss did not improve


Epoch 25/100
Epoch 00025: val_loss did not improve
 - 5s - loss: 0.3273 - acc: 0.8500 - val_loss: 0.2895 - val_acc: 0.8653
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 5s - loss: 0.3275 - acc: 0.8616 - val_loss: 0.3029 - val_acc: 0.8653
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 6s - loss: 0.3311 - acc: 0.8739 - val_loss: 0.2568 - val_acc: 0.8853
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 5s - loss: 0.3232 - acc: 0.8617 - val_loss: 0.3653 - val_acc: 0.8204
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 5s - loss: 0.2612 - acc: 0.8783 - val_loss: 0.2412 - val_acc: 0.8828
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 5s - loss: 0.3043 - acc: 0.8675 - val_loss: 0.2246 - val_acc: 0.9102
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 5s - loss: 0.2930 - acc: 0.8720 - val_loss: 0.2260 - val_acc: 0.9052
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 5s - loss: 0.2785 - acc: 0.8708 - val_loss: 0.2296 - val_acc: 0.8953
Epoch 33/100
Epo

Epoch 88/100
Epoch 00088: val_loss did not improve
 - 5s - loss: 0.2094 - acc: 0.9092 - val_loss: 0.1946 - val_acc: 0.9177
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 5s - loss: 0.2287 - acc: 0.9000 - val_loss: 0.2004 - val_acc: 0.9227
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 5s - loss: 0.2124 - acc: 0.9192 - val_loss: 0.2029 - val_acc: 0.9227
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 5s - loss: 0.2102 - acc: 0.9158 - val_loss: 0.1968 - val_acc: 0.9277
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 5s - loss: 0.2003 - acc: 0.9217 - val_loss: 0.2179 - val_acc: 0.9052
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 5s - loss: 0.2214 - acc: 0.9033 - val_loss: 0.2205 - val_acc: 0.9102
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 5s - loss: 0.2208 - acc: 0.9042 - val_loss: 0.1943 - val_acc: 0.9277
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 5s - loss: 0.2138 - acc: 0.9067 - val_loss: 0.1936 - val_acc: 0.9227
Epoch 96/100
Epo

Epoch 50/100
Epoch 00050: val_loss improved from 0.23313 to 0.22957, saving model to best_m.h5
 - 5s - loss: 0.2534 - acc: 0.8906 - val_loss: 0.2296 - val_acc: 0.9252
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 5s - loss: 0.2539 - acc: 0.8925 - val_loss: 0.2370 - val_acc: 0.9252
Epoch 52/100
Epoch 00052: val_loss improved from 0.22957 to 0.22683, saving model to best_m.h5
 - 5s - loss: 0.2185 - acc: 0.9058 - val_loss: 0.2268 - val_acc: 0.9177
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 5s - loss: 0.2374 - acc: 0.8967 - val_loss: 0.2276 - val_acc: 0.9177
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 5s - loss: 0.2390 - acc: 0.9092 - val_loss: 0.2410 - val_acc: 0.9077
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 5s - loss: 0.2681 - acc: 0.8942 - val_loss: 0.2371 - val_acc: 0.9152
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 5s - loss: 0.2259 - acc: 0.9133 - val_loss: 0.2391 - val_acc: 0.9152
Epoch 57/100
Epoch 00057: val_loss did not improve


Epoch 13/100
Epoch 00013: val_loss improved from 0.27998 to 0.26864, saving model to best_m.h5
 - 5s - loss: 0.3778 - acc: 0.8267 - val_loss: 0.2686 - val_acc: 0.8454
Epoch 14/100
Epoch 00014: val_loss improved from 0.26864 to 0.25915, saving model to best_m.h5
 - 5s - loss: 0.3273 - acc: 0.8564 - val_loss: 0.2592 - val_acc: 0.8778
Epoch 15/100
Epoch 00015: val_loss improved from 0.25915 to 0.23662, saving model to best_m.h5
 - 5s - loss: 0.3488 - acc: 0.8414 - val_loss: 0.2366 - val_acc: 0.8878
Epoch 16/100
Epoch 00016: val_loss did not improve
 - 5s - loss: 0.3532 - acc: 0.8441 - val_loss: 0.2523 - val_acc: 0.8778
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 5s - loss: 0.3824 - acc: 0.8322 - val_loss: 0.3273 - val_acc: 0.9002
Epoch 18/100
Epoch 00018: val_loss did not improve
 - 5s - loss: 0.3485 - acc: 0.8481 - val_loss: 0.2863 - val_acc: 0.8903
Epoch 19/100
Epoch 00019: val_loss improved from 0.23662 to 0.20246, saving model to best_m.h5
 - 5s - loss: 0.3203 - acc: 0.8650 

Epoch 76/100
Epoch 00076: val_loss did not improve
 - 5s - loss: 0.2202 - acc: 0.9117 - val_loss: 0.1902 - val_acc: 0.9127
Epoch 77/100
Epoch 00077: val_loss did not improve
 - 5s - loss: 0.2531 - acc: 0.9045 - val_loss: 0.2005 - val_acc: 0.9277
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 5s - loss: 0.2022 - acc: 0.9183 - val_loss: 0.1941 - val_acc: 0.9327
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 5s - loss: 0.2435 - acc: 0.8884 - val_loss: 0.1900 - val_acc: 0.9401
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 6s - loss: 0.2256 - acc: 0.8992 - val_loss: 0.2016 - val_acc: 0.9377
Epoch 81/100
Epoch 00081: val_loss improved from 0.17928 to 0.17882, saving model to best_m.h5
 - 5s - loss: 0.2299 - acc: 0.9064 - val_loss: 0.1788 - val_acc: 0.9327
Epoch 82/100
Epoch 00082: val_loss improved from 0.17882 to 0.17762, saving model to best_m.h5
 - 5s - loss: 0.2013 - acc: 0.9200 - val_loss: 0.1776 - val_acc: 0.9327
Epoch 83/100
Epoch 00083: val_loss did not improve


In [9]:
with open('../features/resnet_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug2_sub.csv', index=False)

# pre 2271
# new 2118

0.211888493861
         id  is_iceberg
0  5941774d    0.136547
1  4023181e    0.462604
2  b20200e4    0.002311
3  e7f018bb    0.962127
4  4371c8c3    0.032308
